In [1]:
# 최신 노선 정보 불러오기
import pandas as pd
import requests
import json
import xmltodict
import pickle

In [2]:
info = pd.read_csv(r"C:\Users\0811m\Desktop\DScover\메인프로젝트\서울시노선기본정보_수정.csv")


station_info = pd.read_csv(r"C:\Users\0811m\Desktop\DScover\메인프로젝트\arsIds.csv")

# routeID 정보 리스트
route_ids = list(info['ROUTE_ID'].unique())
route_nms = list(info["ROUTE_ABRV"].unique())
station_ars = list(station_info["arsId"].unique().astype(int).astype(str))

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_25140\2067745207.py:7: RuntimeWarning: invalid value encountered in cast
  station_ars = list(station_info["arsId"].unique().astype(int).astype(str))


In [3]:
station_ars

['19757',
 '6282',
 '2699',
 '15309',
 '6001',
 '16921',
 '24279',
 '38182',
 '16301',
 '11182',
 '20123',
 '18960',
 '19434',
 '36693',
 '12126',
 '23360',
 '6585',
 '2502',
 '12294',
 '36055',
 '57014',
 '61102',
 '20515',
 '20890',
 '19839',
 '20808',
 '22112',
 '3211',
 '4112',
 '14340',
 '10233',
 '16140',
 '23538',
 '25193',
 '21244',
 '20725',
 '25558',
 '2506',
 '7393',
 '22131',
 '16214',
 '20675',
 '8517',
 '19802',
 '1157',
 '22520',
 '17954',
 '12573',
 '38629',
 '2008',
 '36298',
 '60122',
 '8881',
 '8459',
 '10510',
 '12811',
 '35031',
 '22808',
 '24002',
 '4849',
 '23130',
 '14756',
 '25589',
 '17477',
 '13142',
 '10129',
 '18565',
 '21792',
 '14181',
 '8157',
 '15172',
 '6012',
 '13941',
 '9590',
 '5757',
 '24550',
 '6302',
 '14275',
 '8160',
 '60017',
 '63105',
 '20937',
 '10145',
 '20186',
 '14898',
 '12156',
 '20703',
 '47156',
 '22766',
 '4767',
 '14560',
 '1843',
 '3194',
 '23962',
 '13012',
 '10240',
 '22867',
 '10141',
 '7406',
 '21271',
 '63252',
 '18531',
 '318

In [4]:
# 입력값
servicekey = '8PsFK0iz82VJUxQTGqiw6Vk14oS0o7OjcAeEG70zLh3sNrzauJ74rtUl4IZN18HtToyjLitr%2Fpeqi95bnL0dLg%3D%3D'

# # 개별 확인용
# route_id = '100100022'
# route_nm = '0017'
url = f'http://ws.bus.go.kr/api/rest/stationinfo/getRouteByStation?ServiceKey={servicekey}&arsId=23231'
response = requests.get(url)
contents = response.text

# xml을 json으로 변경
data_dict = xmltodict.parse(contents)
# data_dict

print(contents)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><ServiceResult><comMsgHeader/><msgHeader><headerCd>0</headerCd><headerMsg>정상적으로 처리되었습니다.</headerMsg><itemCount>0</itemCount></msgHeader><msgBody><itemList><busRouteAbrv>강남10</busRouteAbrv><busRouteId>122900009</busRouteId><busRouteNm>강남10</busRouteNm><busRouteType>2</busRouteType><firstBusTm>20200302000000</firstBusTm><firstBusTmLow>              </firstBusTmLow><lastBusTm>20160601000000</lastBusTm><lastBusTmLow>              </lastBusTmLow><length>16.3</length><nextBus>10</nextBus><stBegin>개포주공7단지앞</stBegin><stEnd>양재역</stEnd><term>14</term></itemList><itemList><busRouteAbrv>143</busRouteAbrv><busRouteId>100100022</busRouteId><busRouteNm>143</busRouteNm><busRouteType>3</busRouteType><firstBusTm>20150717000000</firstBusTm><firstBusTmLow>20190222000000</firstBusTmLow><lastBusTm>20150717000000</lastBusTm><lastBusTmLow>20190222000000</lastBusTmLow><length>62.2</length><nextBus>10</nextBus><stBegin>정릉</stBegin><stEnd>개포동</stEnd><term>6<

In [6]:
print(data_dict['ServiceResult']['msgBody']['itemList'])

[{'busRouteAbrv': '1127', 'busRouteId': '100100143', 'busRouteNm': '1127', 'busRouteType': '4', 'firstBusTm': '20150717000000', 'firstBusTmLow': None, 'lastBusTm': '20150717000000', 'lastBusTmLow': None, 'length': '20', 'nextBus': '10', 'stBegin': '수유역', 'stEnd': '도봉산', 'term': '14'}, {'busRouteAbrv': '1128', 'busRouteId': '100100144', 'busRouteNm': '1128', 'busRouteType': '4', 'firstBusTm': '20151224000000', 'firstBusTmLow': None, 'lastBusTm': '20151224000000', 'lastBusTmLow': None, 'length': '30.5', 'nextBus': '10', 'stBegin': '도봉산', 'stEnd': '월곡동', 'term': '12'}]


In [5]:

api_dict = {}

for arsid in station_ars:
    def fetch_data(arsid):
        # API로부터 데이터 요청
        url = f'http://ws.bus.go.kr/api/rest/stationinfo/getRouteByStation?ServiceKey={servicekey}&arsId={arsid}'
        response = requests.get(url)
        contents = response.text

        # XML을 JSON으로 변경
        data_dict = xmltodict.parse(contents)
        return data_dict

    # 데이터 요청 시도
    data_dict = fetch_data(arsid)

    # 결과가 없는 경우, arsid의 앞자리에 0을 붙여서 다시 시도
    if data_dict['ServiceResult']['msgHeader']['headerMsg'] == '결과가 없습니다.':
        arsid = '0' + arsid
        data_dict = fetch_data(arsid)

    # itemList 추출
    try:
        item_list = data_dict['ServiceResult']['msgBody']['itemList']
    except (TypeError, KeyError):
        print(f"Error: Cannot extract itemList for station {arsid}")
        continue

    # 각 항목을 데이터프레임으로 변환하여 리스트에 추가
    dfs = []
    for item in item_list:
        df = pd.DataFrame([item])
        dfs.append(df)

    # 리스트에 있는 모든 데이터프레임을 합쳐서 하나의 데이터프레임으로 만듦
    combined_df = pd.concat(dfs, ignore_index=True)

    # 노선명을 키로 하여 딕셔너리에 추가
    api_dict[arsid] = combined_df


Error: Cannot extract itemList for station 010233
Error: Cannot extract itemList for station 012573
Error: Cannot extract itemList for station 010240
Error: Cannot extract itemList for station 01181
Error: Cannot extract itemList for station 016337
Error: Cannot extract itemList for station 04581
Error: Cannot extract itemList for station 022398
Error: Cannot extract itemList for station 01193
Error: Cannot extract itemList for station 01122
Error: Cannot extract itemList for station 01233
Error: Cannot extract itemList for station 018953
Error: Cannot extract itemList for station 01282
Error: Cannot extract itemList for station 011564
Error: Cannot extract itemList for station 018956
Error: Cannot extract itemList for station 016334
Error: Cannot extract itemList for station 01195
Error: Cannot extract itemList for station 7616
Error: Cannot extract itemList for station 7109
Error: Cannot extract itemList for station 48641
Error: Cannot extract itemList for station 16519
Error: Cannot

KeyboardInterrupt: 

In [36]:
# 딕셔너리 저장
import pickle
# save data
with open(r'C:\Users\0811m\Desktop\DScover\메인프로젝트\getRouteByStation_143.pickle','wb') as fw:
    pickle.dump(api_dict, fw)